In [1]:
import glob
import os
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F

In [2]:
%matplotlib inline

In [3]:
is_cuda = torch.cuda.is_available()

In [4]:
transformation = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.1307, ), (0.3081, ))
    ]
)

In [5]:
# train_dataset = torchvision.datasets.MNIST('data/', train=True, download=True, transform=transformation)
# test_dataset = torchvision.datasets.MNIST('data/', train=False, download=True, transform=transformation)

In [6]:
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)

In [7]:
#train_loader_iter = train_loader.__iter__()

In [8]:
def plot_img(image):
    image = image.numpy()[0]
    mean = 0.1307
    std = 0.3081
    image = ((mean * image) + std)
    plt.imshow(image, cmap='gray')

In [9]:
#sample_data = train_loader_iter.__next__()

In [10]:
#plot_img(sample_data[0][1])

In [11]:
#plot_img(sample_data[0][2])

In [12]:
# class Net(torch.nn.Module):
    
#     def __init__(self):
#         super().__init__()
#         self.conv1 = torch.nn.Conv2d(1, 10, kernel_size=5)
#         self.conv2 = torch.nn.Conv2d(10, 20, kernel_size=5)
#         self.conv2_drop = torch.nn.Dropout2d()
#         self.fc1 = torch.nn.Linear(320, 50)
#         self.fc2 = torch.nn.Linear(50, 10)
        
#     def forward(self, x):
#         x = F.relu(F.max_pool2d(self.conv1(x), 2))
#         x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
#         x = x.view(-1, 320)
#         x = F.relu(self.fc1(x))
#         x = F.dropout(x, training=self.training)
#         x = self.fc2(x)
#         return F.log_softmax(x)

In [13]:
# class Net(torch.nn.Module):
    
#     def __init__(self):
#         super().__init__()
#         self.conv1 = torch.nn.Conv2d(3, 10, kernel_size=5)
#         self.conv2 = torch.nn.Conv2d(10, 20, kernel_size=5)
#         self.conv2_drop = torch.nn.Dropout2d()
#         self.fc1 = torch.nn.Linear(56180, 500)
#         #self.fc1 = torch.nn.Linear(67840, 53)
#         self.fc2 = torch.nn.Linear(500, 50)
#         #self.fc2 = torch.nn.Linear(53, 50)
#         self.fc3 = torch.nn.Linear(50, 2)
    
#     def forward(self, x):
#         x = F.relu(F.max_pool2d(self.conv1(x), 2))
#         x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
#         x = x.view(x.size(0), -1)
#         x = F.relu(self.fc1(x))
#         x = F.dropout(x, training=self.training)
#         x = F.relu(self.fc2(x))
#         x = F.dropout(x, training=self.training)
#         x = self.fc3(x)
#         return F.log_softmax(x, dim=1)

In [14]:
#conv = torch.nn.Conv1d(1, 1, 3, bias=False)

In [15]:
#sample = torch.randn(1, 1, 7)

In [16]:
#sample

In [17]:
#conv(Variable(sample))

In [18]:
#conv.weight

In [19]:
# simple_transform = transforms.Compose(
#     [
#         transforms.Scale((224, 224)),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]
# )

In [20]:
train_transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]
)

In [21]:
#train = torchvision.datasets.ImageFolder('chapter3/dogsandcats/train/', simple_transform)
#valid = torchvision.datasets.ImageFolder('chapter3/dogsandcats/valid/', simple_transform)
train = torchvision.datasets.ImageFolder('chapter3/dogsandcats/train/', train_transform)
valid = torchvision.datasets.ImageFolder('chapter3/dogsandcats/valid/', train_transform)

In [22]:
train_data_gen = torch.utils.data.DataLoader(train, batch_size=64, num_workers=8)
valid_data_gen = torch.utils.data.DataLoader(valid, batch_size=64, num_workers=8)

In [23]:
train_loader = train_data_gen
test_loader = valid_data_gen

In [24]:
is_cuda

True

In [25]:
def fit(epoch, model, data_loader, phase='training', volatile=False):
    if is_cuda:
        model = model.cuda()
    if phase == 'training':      
        model.train()
    if phase == 'validation':
        model.eval()
        volatile=True
    running_loss = 0.0
    running_correct = 0
    for batch_idx, (data, target) in enumerate(data_loader):
        if is_cuda:
            data = data.cuda()
            target = target.cuda()
        data = Variable(data, volatile)
        target = Variable(target)
        if phase == 'training':
            optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        running_loss += F.nll_loss(output, target, size_average=False).data[0]
        #running_loss += (F.nll_loss(output, target, size_average=False)).item()
        print("running_loss: ", running_loss)
        preds = output.data.max(dim=1, keepdim=True)[1]
        print("preds:\n", preds)
        running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
        if phase == 'training':
            loss.backward()
            optimizer.step()
    loss = running_loss / len(data_loader.dataset)
    accuracy = 100. * running_correct / len(data_loader.dataset)
    print(
        f"{phase}, loss is {loss:{5}.{2}} and {phase} accuracy is {running_correct} / {len(data_loader.dataset)}"
        #" {accuracy:{10}.{4}}")
        "  {}".format(accuracy))
    return loss, accuracy 

In [26]:
#model = Net()

In [27]:
#if is_cuda:
#    model = model.cuda()

In [28]:
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
# train_losses = []
# train_accuracy = []
# val_losses = []
# val_accuracy = []
# for epoch in range(1, 20):
#     epoch_loss, epoch_accuracy = fit(epoch, model, train_loader, phase='training')
#     val_epoch_loss, val_epoch_accuracy = fit(epoch, model, test_loader, phase='validation')
#     train_losses.append(epoch_loss)
#     train_accuracy.append(epoch_accuracy)
#     val_losses.append(val_epoch_loss)
#     val_accuracy.append(val_epoch_accuracy)

In [29]:
# plt.plot(range(1, len(train_losses) + 1), train_losses, 'bo', label="training loss")
# plt.plot(range(1, len(val_losses) + 1), val_losses, 'r', label="validation loss")
# plt.legend()

In [30]:
# plt.plot(range(1, len(train_accuracy) + 1), train_accuracy, 'bo', label="training accuracy")
# plt.plot(range(1, len(val_accuracy) + 1), val_accuracy, 'r', label="validation accuracy")
# plt.legend()

In [ ]:
vgg = torchvision.models.vgg16(pretrained=True)

In [ ]:
vgg

In [ ]:
for param in vgg.features.parameters():
    param.requires_grad = False

In [ ]:
vgg.classifier[6].out_features = 2

In [ ]:
optimizer = torch.optim.SGD(vgg.classifier.parameters(), lr=0.001, momentum=0.5)

In [ ]:
for layer in vgg.children():
    if type(layer) == torch.nn.Dropout:
        layer.p = 0.2

In [ ]:
train_losses = []
train_accuracy = []
val_losses = []
val_accuracy = []
for epoch in range(1, 10):
    epoch_loss, epoch_accuracy = fit(epoch, vgg, train_loader, phase='training')
    val_epoch_loss, val_epoch_accuracy = fit(epoch, vgg, test_loader, phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)

In [ ]:
is_cuda